In [ ]:
import os
import json
import pandas as pd
import traceback

In [ ]:
from langchain_community.chat_models import ChatOpenAI

In [ ]:
from dotenv import load_dotenv
load_dotenv() # take environment variables from .env

In [ ]:
KEY=os.getenv("OPENAI_API_KEY") # Creating the environment variable

In [ ]:
# Creating the object of the class ChatOpenAI
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5) # temperature ranges from 0 to 2. If you are mentioning the value near to 2, then the model will be more creative and if it is near to 0 then the model will be less creative.

In [ ]:
llm

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain # LLMChain is used to connect several components
from langchain.chains import SequentialChain
from langchain_community.callbacks import get_openai_callback
# from langchain.callbacks import get_openai_callback
import PyPDF2

In [ ]:
# Response Format
RESPONSE_JSON={
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [ ]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [ ]:
# Define Prompt Template
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"], # These are the variables which user is going to pass
    template=TEMPLATE
)

In [ ]:
# Creating LLM Chain
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [ ]:
# Template for correct answer
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
# Defining second prompt template
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [ ]:
# Creating second LLM chain
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True) # verbose means whatever execution we are doing we will find on the screen itself

In [ ]:
# Connecting both the LLM Chain with the help of Sequential Chain
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                                        output_variables=["quiz","review"],verbose=True)

In [ ]:
file_path=r"D:\Projects\MCQGEN\data.txt" # r means read it

In [ ]:
file_path

In [ ]:
with open(file_path, 'r') as file:
    TEXT=file.read()

In [ ]:
print(TEXT)

In [ ]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

In [ ]:
# These is the only thing that I want from my user. So for these we will create web application
NUMBER=5 # Number means how many quiz we have to generate
SUBJECT="machine learning" # Name of the subject
TONE="simple" # Tone means style of the tone for giving the answer

In [ ]:
# https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

# How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

In [ ]:
response

In [ ]:
quiz=response.get("quiz")

In [ ]:
quiz=json.loads(quiz)

In [ ]:
quiz_table_data=[]
for key, value in quiz.items(): # here value means options
    mcq=value["mcq"]
    options=" | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct=value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
# Converting it into dataframe
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
# Converting the dataframe as a csv file
quiz.to_csv("machinelearning.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')